<center><h2>
<a href="https://masumbhai.me">Kindly visit my portfolio to see more of my works</a>
</h2></center>

In [27]:
import warnings

import pandas as pd
from dask import dataframe as dd
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
imgPath = "G:\\Brig_Gen_Razzak_Sir_Thesis_Group\\Thesis-on-DDOS-main\\images\\"

In [2]:
def readCSV_Function(file_path):
    dask_df = dd.read_csv(file_path, blocksize=1e6)
    dask_df = dask_df.repartition(npartitions=8)
    return dask_df.compute(scheduler='threads')

In [3]:
filePath = "D:\\Thesis_Group_of_Brig_Gen_Razzak_Sir\\After_Column_Data_Cleaning\\"
fileName = "after_dropping_unique_data_containing_features.csv"

initial_df = readCSV_Function(filePath+fileName)
initial_df.drop(columns=["Unnamed: 0"], inplace=True, errors='ignore')
initial_df = shuffle(initial_df)
initial_df.shape

(299880, 66)

In [4]:
features_filepath = "D:\\Thesis_Group_of_Brig_Gen_Razzak_Sir\\After_Feature_Engineering\\Selected_Features_along_with_PCA.csv"
features_df = readCSV_Function(features_filepath)
features_df.drop(columns=["Unnamed: 0"], inplace=True, errors='ignore')
features_df.shape

(43, 2)

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

final_selected_features = sorted(list(features_df["Names"]))
df = initial_df[final_selected_features]

weights = df['Label'].value_counts()/len(df)
y = df['Label']
X = df.drop(columns='Label')

# X_std = StandardScaler().fit_transform(X) # data standardized / scaled here
# X_norm = preprocessing.normalize(X_std)
#
# X_train, X_test, y_train, y_test = train_test_split(X, y ,
#                                                     random_state=1,
#                                                     test_size=0.2,
#                                                     shuffle=True)

<center><h3 style="background:#00FF00;color:black">
Saving the imbalanced initial dataset
</h3></center>

In [6]:
filePath = "D:\\Thesis_Group_of_Brig_Gen_Razzak_Sir\\For_Model_Creation\\"
fileName = "imbalanced_43_features_DDOS_dataset.csv"
df.to_csv(filePath+fileName)

<center><h3 style="background:#FFEB3B;color:black">
Applying Condensed Nearest Neighbors undersampling
</h3></center>

In [ ]:
from collections import Counter
from imblearn.under_sampling import CondensedNearestNeighbour

cnn = CondensedNearestNeighbour(n_neighbors=5,n_jobs=-1,random_state=42)
X_resampled_cnn, y_resampled_cnn = cnn.fit_resample(X, y)

print("Resampled class distribution:", Counter(y_resampled_cnn))

In [ ]:
imgName = "Undersampled_by_Condensed_Nearest_Neighbour.png"
counter = Counter(y_resampled_cnn)
colors = ["#33FF33","#03A9F4","magenta","#F4A460","#0099FF","#FFEB3B","#C0C0C0","orange","#FF0099","#FF6600","#9B59B6"]

plt.figure(figsize=(30,10))
plt.bar(x=counter.keys(),height=counter.values(),color=colors,alpha=0.7,align="center",linewidth=0.5)
plt.rcParams['font.size'] = 24
plt.title("Condensed Nearest Neighbour Under-sampling")
plt.tight_layout()
plt.savefig(imgPath+imgName)

<center><h3 style="background:#FFEB3B;color:black">
Applying Edited Nearest Neighbors undersampling
</h3></center>

In [10]:
from collections import Counter
from imblearn.under_sampling import EditedNearestNeighbours

enn = EditedNearestNeighbours()
X_resampled_enn, y_resampled_enn = enn.fit_resample(X, y)

print("Resampled class distribution:", Counter(y_resampled_enn))

Resampled class distribution: Counter({'TFTP': 29399, 'Syn': 26679, 'DrDoS_NTP': 21740, 'DrDoS_MSSQL': 21375, 'UDP-lag': 19164, 'DrDoS_SNMP': 17288, 'DrDoS_LDAP': 11001, 'DrDoS_DNS': 9842, 'BENIGN': 4670, 'DrDoS_UDP': 3691, 'DrDoS_SSDP': 3370})


In [ ]:
imgName = "Undersampled_by_Edited_Nearest_Neighbour.png"
counter = Counter(y_resampled_enn)
colors = ["#33FF33","#03A9F4","magenta","#F4A460","#0099FF","#FFEB3B","#C0C0C0","orange","#FF0099","#FF6600","#9B59B6"]

plt.figure(figsize=(30,10))
plt.bar(x=counter.keys(),height=counter.values(),color=colors,alpha=0.7,align="center",linewidth=0.5)
plt.rcParams['font.size'] = 24
plt.title("Edited Nearest Neighbour Under-sampling")
plt.tight_layout()
plt.savefig(imgPath+imgName)

<center><h3 style="background:#FFEB3B;color:black">
Applying Near Miss version-1 undersampling
</h3></center>

In [16]:
from imblearn.under_sampling import NearMiss
from collections import Counter

nm1 = NearMiss(version=1)
X_resampled_nm1, y_resampled_nm1 = nm1.fit_resample(X, y)

print("Resampled class distribution:", Counter(y_resampled_nm1))

Resampled class distribution: Counter({'BENIGN': 4670, 'DrDoS_DNS': 4670, 'DrDoS_LDAP': 4670, 'DrDoS_MSSQL': 4670, 'DrDoS_NTP': 4670, 'DrDoS_SNMP': 4670, 'DrDoS_SSDP': 4670, 'DrDoS_UDP': 4670, 'Syn': 4670, 'TFTP': 4670, 'UDP-lag': 4670})


In [ ]:
imgName = "Undersampled_by_Near_Miss_Version-1.png"
counter = Counter(y_resampled_nm1)
colors = ["#33FF33","#03A9F4","magenta","#F4A460","#0099FF","#FFEB3B","#C0C0C0","orange","#FF0099","#FF6600","#9B59B6"]

plt.figure(figsize=(30,10))
plt.bar(x=counter.keys(),height=counter.values(),color=colors,alpha=0.7,align="center",linewidth=0.5)
plt.rcParams['font.size'] = 24
plt.title("Near Miss Version-1 Under-sampling")
plt.tight_layout()
plt.savefig(imgPath+imgName)

<center><h3 style="background:#FFEB3B;color:black">
Applying Near Miss version-2 undersampling
</h3></center>

In [15]:
from imblearn.under_sampling import NearMiss
from collections import Counter

nm2 = NearMiss(version=2)
X_resampled_nm2, y_resampled_nm2 = nm2.fit_resample(X, y)

print("Resampled class distribution:", Counter(y_resampled_nm2))

Resampled class distribution: Counter({'BENIGN': 4670, 'DrDoS_DNS': 4670, 'DrDoS_LDAP': 4670, 'DrDoS_MSSQL': 4670, 'DrDoS_NTP': 4670, 'DrDoS_SNMP': 4670, 'DrDoS_SSDP': 4670, 'DrDoS_UDP': 4670, 'Syn': 4670, 'TFTP': 4670, 'UDP-lag': 4670})


In [ ]:
imgName = "Undersampled_by_Near_Miss_Version-2.png"
counter = Counter(y_resampled_nm2)
colors = ["#33FF33","#03A9F4","magenta","#F4A460","#0099FF","#FFEB3B","#C0C0C0","orange","#FF0099","#FF6600","#9B59B6"]

plt.figure(figsize=(30,10))
plt.bar(x=counter.keys(),height=counter.values(),color=colors,alpha=0.7,align="center",linewidth=0.5)
plt.rcParams['font.size'] = 24
plt.title("Near Miss Version-2 Under-sampling")
plt.tight_layout()
plt.savefig(imgPath+imgName)

<center><h3 style="background:#FFEB3B;color:black">
Applying Cluster Centroids undersampling
</h3></center>

In [14]:
from collections import Counter
from imblearn.under_sampling import ClusterCentroids

cc_US = ClusterCentroids(sampling_strategy='majority')
X_resampled_cc, y_resampled_cc = cc_US.fit_resample(X, y)

print("Resampled class distribution:", Counter(y_resampled_cc))

Resampled class distribution: Counter({'TFTP': 29958, 'DrDoS_UDP': 29950, 'Syn': 29915, 'DrDoS_MSSQL': 29704, 'DrDoS_NTP': 29678, 'DrDoS_SNMP': 29497, 'DrDoS_DNS': 29204, 'UDP-lag': 28720, 'DrDoS_LDAP': 28618, 'BENIGN': 4670, 'DrDoS_SSDP': 4670})


In [ ]:
imgName = "Undersampled_by_Cluster_Centroids.png"
counter = Counter(y_resampled_cc)
colors = ["#33FF33","#03A9F4","magenta","#F4A460","#0099FF","#FFEB3B","#C0C0C0","orange","#FF0099","#FF6600","#9B59B6"]

plt.figure(figsize=(30,10))
plt.bar(x=counter.keys(),height=counter.values(),color=colors,alpha=0.7,align="center",linewidth=0.5)
plt.rcParams['font.size'] = 24
plt.title("Cluster Centroids Under-sampling")
plt.tight_layout()
plt.savefig(imgPath+imgName)

<center><h3 style="background:#FF6F00;color:black">
Applying SMOTE (Synthetic Minority Oversampling)
</h3></center>

In [17]:
from collections import Counter
from imblearn.over_sampling import SMOTE

sm = SMOTE(k_neighbors=3,random_state = 42)
X_resampled_sm, y_resampled_sm = sm.fit_resample(X, y)

print("Resampled class distribution:", Counter(y_resampled_sm))

Resampled class distribution: Counter({'DrDoS_SNMP': 29966, 'DrDoS_SSDP': 29966, 'TFTP': 29966, 'Syn': 29966, 'DrDoS_UDP': 29966, 'DrDoS_NTP': 29966, 'UDP-lag': 29966, 'DrDoS_DNS': 29966, 'DrDoS_MSSQL': 29966, 'BENIGN': 29966, 'DrDoS_LDAP': 29966})


In [ ]:
imgName = "Oversampled_by_SMOTE.png"
counter = Counter(y_resampled_sm)
colors = ["#33FF33","#03A9F4","magenta","#F4A460","#0099FF","#FFEB3B","#C0C0C0","orange","#FF0099","#FF6600","#9B59B6"]

plt.figure(figsize=(30,10))
plt.bar(x=counter.keys(),height=counter.values(),color=colors,alpha=0.7,align="center",linewidth=0.5)
plt.rcParams['font.size'] = 24
plt.title("SMOTE (Synthetic Minority) Over-sampling")
plt.tight_layout()
plt.savefig(imgPath+imgName)

<center><h3 style="background:#FF6F00;color:black">
Applying BorderLine SMOTE Oversampling
</h3></center>

In [25]:
from collections import Counter
from imblearn.over_sampling import BorderlineSMOTE

blsm = BorderlineSMOTE(random_state=42)
X_resampled_blsm, y_resampled_blsm = blsm.fit_resample(X, y)

print("Resampled class distribution:", Counter(y_resampled_blsm))

Resampled class distribution: Counter({'DrDoS_SNMP': 29966, 'DrDoS_SSDP': 29966, 'TFTP': 29966, 'Syn': 29966, 'DrDoS_UDP': 29966, 'DrDoS_NTP': 29966, 'UDP-lag': 29966, 'DrDoS_DNS': 29966, 'DrDoS_MSSQL': 29966, 'BENIGN': 29966, 'DrDoS_LDAP': 29966})


In [ ]:
imgName = "Oversampled_by_BorderLine_SMOTE.png"
counter = Counter(y_resampled_blsm)
colors = ["#33FF33","#03A9F4","magenta","#F4A460","#0099FF","#FFEB3B","#C0C0C0","orange","#FF0099","#FF6600","#9B59B6"]

plt.figure(figsize=(30,10))
plt.bar(x=counter.keys(),height=counter.values(),color=colors,alpha=0.7,align="center",linewidth=0.5)
plt.rcParams['font.size'] = 24
plt.title("BorderLine SMOTE (Synthetic Minority) Over-sampling")
plt.tight_layout()
plt.savefig(imgPath+imgName)

<center><h3 style="background:#4FC3F7;color:black">
Bar Chart of "Label" Feature or target variable
</h3></center>

# Summarize distribution
counter = Counter(y_resampled)
for i,j in counter.items():
    per = j/len(y_resampled) * 100
    print(f"Label:{i}  n:{j}  {per:.3}%")